**Instituto Tecnológico de Costa Rica**  
**Maestría en Electrónica**  
**MP6124. Procesamiento Adaptativo**    
**Profesor:** Juan Pablo Soto Quirós  
**Estudiantes:**  
- Juan Carlos Cruz Naranjo  
- Luis Alonso Murillo Rojas

<center>$\textbf{Tarea 2}$</center>

#### Problema 1.
Del libro _Fundamentals of Adaptive Filtering_ de Ali Sayed, implemente cada uno de los algoritmos de
gradiente estocástico que se encuentran en la **Tabla 5.1.**, página 230. Cada uno de los algoritmos de la Tabla 5.1. da una aproximación a la solución del problema de optimización:  

<center>$\underset{w \in \mathbb{C}^{n}}{\text{min}}\mathbb{E}[|\textbf{d-u}w|^{2}]$</center>

donde $\textbf{d}$ es una variable aleatoria, $\textbf{u}$ es un vector fila aleatorio de tamaño _n_ y $w$ es un vector columna constante de tamaño _n_. Para la implementación de cada algoritmo, considere las siguientes indicaciones:
- Cada algoritmo debe ser implementado como una función.
- Los parámetros iniciales de cada función son los siguientes:
 - Observaciones de la variable aleatoria $\textbf{d}$ que se representan en el vector $d=[d_0\text{ }d_1\text{ }...\text{ }d_{s−1}]\in \mathbb{C}^{s}$, donde $d_j \in \mathbb{C}$ es la observación $j$ de $\textbf{d}$.  
 - Observaciones del vector aleatorio $\textbf{u}$, que se representan en la matriz $U= 
\begin{bmatrix} 
u_0 \\
\vdots \\
u_{s-1}
\end{bmatrix} \in \mathbb{C}^{s\times n}$, donde $u_j \in \mathbb{C}^{n}$ es la observación $j$ de $\textbf{u}$.

 - Vector inicial $w^{(−1)} \in \mathbb{C}^m$, el cual es un vector columna.
 - Tolerancia $tol > 0$ para el criterio de parada.
 - Iteraciones máximas $iterMax > 0$.
- Los parámetros finales son el número de iteraciones $k$ y el vector $w^{(k)}$ que aproxima la solución del problema (1).
- Utilice nombres significativos para definir cada una de las funciones.
- El criterio de parada de cada algoritmo será la condición $||w^{(k)} - w^{(k−1)}||_2 < tol$ o cuando el número de iteraciones es mayor a $iterMax$.

- En el método LMS con tamaño de paso variante, puede utilizar como tamaño de paso $\mu_i = \frac{2}{1+i}$ en cada iteración. Si considera oportuno, puede utilizar otro tamaño de paso. De ser así, debe justificar porque seleccionó otro tamaño de paso $\mu_i$.
- Cada método debe generar dos gráficas: una de iteraciones versus error y otra de iteraciones versus valor mínimo. El error se medirá con el criterio $e_k = ||w^{(k)} - w^{(k−1)}||_2$ y el valor mínimo se medirá con el criterio:

<center>$m_k = \hat{R}_{d_kd_k}-\left(\hat{R}_{d_ku_k}\right)^{*}w^{(k)}-\left(w^{(k)}\right)^{*}\hat{R}_{d_ku_k} + \left(w^{(k)}\right)^{*}\hat{R}_{u_ku_k}w^{(k)}$</center>


Donde $\hat{R}_{d_kd_k}=d_k(d_k)^*$, $\hat{R}_{d_ku_k}=d_k(u_k)^*$ y $\hat{R}_{u_ku_k}=(u_k)^*u_k$.